In [27]:
import pandas as pd
import nltk
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from colour import Color
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from IPython.display import display, Markdown


In [28]:
DARK_BLUE = '#3557A9'
DARK_GREEN = '#50974F'
DARK_RED = '#D43F2C'
DARK_YELLOW = '#F8D65A'

BLUE = '#627EC3'
GREEN = '#7ABA79'
RED = '#E36656'
YELLOW = '#FFE480'

LIGHT_BLUE = '#8CA5E2'
LIGHT_GREEN = '#A3D9A3'
LIGHT_RED = '#F6887A'
LIGHT_YELLOW = '#FFEA9D'

## Load Data

In [29]:
import pandas as pd

# Caminhos para os arquivos CSV
sheets_path = 'C:/Users/leibr/Documents/Lappis/data/2023-53345.csv' #sheets 
planilha_path = 'C:/Users/leibr/Documents/Lappis/data/Propostas-Brasil-Participativo-TagsCategorias.csv' #planilha

# Carregar os arquivos CSV
df_sheets = pd.read_csv(sheets_path, sep=';')
df_planilha = pd.read_csv(planilha_path, sep=';')


## Pre-Process Data

In [30]:
# Convertendo a coluna 'ID' em df_planilha para int64
df_planilha['ID'] = df_planilha['ID'].astype(int)

# Mesclando data sheets e planilha
df = pd.merge(df_sheets, df_planilha, left_on='id', right_on='ID', how='left')

# Remover (state = rejected) ou (state = withdrawn)
df = df[df['state'] != 'rejected']
df = df[df['state'] != 'withdrawn']

# Remover html tags from body
df['body/pt-BR'] = df['body/pt-BR'].str.replace(r'<[^>]*>', '', regex=True)

In [18]:
# Convertendo a coluna 'ID' em df_planilha para int64
df_planilha['ID'] = df_planilha['ID'].astype(int)

# Mesclando data sheets e planilha
df = pd.merge(df_sheets, df_planilha, left_on='id', right_on='ID', how='left')

# Remover (state = rejected) ou (state = withdrawn)
df = df[df['state'] != 'rejected']
df = df[df['state'] != 'withdrawn']

# Remover html tags from body
df['body/pt-BR'] = df['body/pt-BR'].str.replace(r'<[^>]*>', '', regex=True)


## Explore Data

In [31]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
df = df[[
    'id',
    'category/name/pt-BR',
    'title/pt-BR',
    'body/pt-BR',
    'supports',
    'followers',
    'published_at',
    # 'related_proposals', # not useful yet
    'ID',
    # 'Tema', # not useful
    # 'Ministério', # not useful
    'Categoria 1',
    'Categoria 2',
    'Título',
    '#CUIDADO', # might be useful
]]
df

,id,category/name/pt-BR,title/pt-BR,body/pt-BR,supports,followers,published_at,ID,Categoria 1,Categoria 2,Título,#CUIDADO
1,5990,Trabalho e Emprego,Progama educação politica,Falar sobre as estruturas do governo e as opor...,3,1,03.07.2023 20:27:03,NaN,NaN,NaN,NaN,NaN
2,3503,"Ciência, Tecnologia e Inovação",Hábitos inclusivos,Financiar mecanismos de adaptação de pessoas c...,4,1,16.06.2023 22:15:30,3503.0,Financiamento,Acessibilidade,Hábitos inclusivos,Sim
4,4,Saúde,Atenção Primária à Saúde,Objetivo: Ampliar o acesso às ações e serviços...,0,0,10.05.2023 19:41:01,4.0,Saúde,Atenção Primária,Atenção Primária à Saúde,Sim
5,6,"Desenvolvimento e Assistência Social, Família ...",Bolsa Família: Proteção Social por Meio da Tra...,Objetivo: Reduzir a pobreza por meio da transf...,0,0,10.05.2023 19:43:27,6.0,Bolsa Família,Sustentabilidade / Baixa Renda,Bolsa Família: Proteção Social por Meio da Tra...,Não
6,7,Comunicações,Brasil Conectado: Comunicações para Inclusão e...,Objetivo: Assegurar serviços de comunicações e...,0,0,10.05.2023 19:44:34,7.0,Inclusão Digital,Acesso à Informação,Brasil Conectado: Comunicações para Inclusão e...,Não
...,...,...,...,...,...,...,...,...,...,...,...,...
7576,145,Saúde,Legalização da cannabis para uso medicinal e r...,"A política de guerra às drogas, que tem origen...",637,43,11.05.2023 15:47:41,145.0,Entorpecentes / Direito Penal e Poder de Polícia,NaN,Legalização da cannabis para uso medicinal e r...,Não
7577,10086,Direitos Humanos e Cidadania,Anistia para os empregados públicos perseguido...,Criar lei de anistia p/ empregados públicos co...,85,8,14.07.2023 12:19:09,NaN,NaN,NaN,NaN,NaN
7578,10304,Justiça e Segurança Pública,"Liberação de compra, possa e porte de armas de...","Propor a liberação da compra, posse e posse de...",1,1,14.07.2023 17:07:16,NaN,NaN,NaN,NaN,NaN
7579,71,Minas e Energia,Reestatização da Eletrobrás,A privatização reduz a segurança energética do...,3390,321,11.05.2023 12:58:24,71.0,Estatização / Eletricidade / Empresas Públicas,Iluminação Pública,Reestatização da Eletrobrás,Não


### Criar arquivo MD

In [53]:
md_filename = 'relatorio.md'

with open(md_filename, 'w', encoding='utf-8') as md_file:
    md_file.write('# Relatório de Análise de Propostas\n\n')
    md_file.write('## Resumo das Propostas Categorizadas\n\n')

In [37]:
# Caminho para o arquivo de saída em formato Markdown
md_filename = 'relatorio.md'

# Abrir o arquivo para escrita
with open(md_filename, 'w') as md_file:

    # Escrever o título e subtítulo
    md_file.write('# Relatório de Análise de Propostas\n\n')
    md_file.write('## Resumo das Propostas Categorizadas\n\n')

    # Gerar o gráfico de pizza
    plt.figure(figsize=(8, 8))
    df['ID'].notna().value_counts().plot.pie(
        autopct=lambda p: f"{p * df.shape[0] / 100 :.0f}\n{p:.0f}%",
        fontsize=15,
        ylabel='',
        labels=['Sim', 'Não'],
        textprops={'color': 'black', 'fontweight': 'semibold'},
        colors=[DARK_BLUE, LIGHT_BLUE],
    )
    plt.title('Propostas Categorizadas Manualmente', fontsize=20, fontweight='bold')
    plt.legend([f'Total: {df.shape[0]:.0f}'], loc=(-0.2, -0.1), fontsize=15, handlelength=0)
    
    # Salvar o gráfico como imagem temporária
    temp_img_filename = 'temp_pie.png'
    plt.savefig(temp_img_filename)
    plt.close()
    
    # Incluir o gráfico no arquivo MD
    md_file.write('### Gráfico de Propostas Categorizadas\n\n')
    md_file.write('![Gráfico de Propostas Categorizadas](' + temp_img_filename + ')\n\n')

    # Escrever o título e a tabela
    md_file.write('## Quantidade de Propostas por Ministério\n\n')
    
    # Gerar a tabela de quantidade de propostas por ministério
    top_propostas = df.groupby('category/name/pt-BR')['category/name/pt-BR'].count().sort_values(ascending=False).head(40)
    md_file.write(top_propostas.to_markdown())

    # Escrever o título e a tabela
    md_file.write('## Quantidade de Votos por Ministério\n\n')
    
    # Gerar a tabela de quantidade de votos por ministério
    top_votos = df.groupby('category/name/pt-BR')['supports'].sum().sort_values(ascending=False).head(40)
    md_file.write(top_votos.to_markdown())

print(f"Relatório Markdown gerado: {md_filename}")

Relatório Markdown gerado: relatorio.md


### ANÁLISE GERAL: Propostas Categorizadas Manualmente

In [54]:
# Código para plotar o gráfico de pizza

plt.figure(figsize=(10, 6))
df['ID'].notna().value_counts().plot.pie(autopct=lambda p: f"{p * df.shape[0] / 100 :.0f}\n{p:.0f}%",
                                         fontsize=15,
                                         ylabel='',
                                         labels=['Sim', 'Não'],
                                         textprops={'color': 'black', 'fontweight': 'semibold'},
                                         colors=[DARK_BLUE, LIGHT_BLUE],
                                         )
plt.title('Propostas Categorizadas Manualmente', fontsize=20, fontweight='bold')
plt.legend([f'Total: {df.shape[0]:.0f}'], loc=(-0.2, -0.1), fontsize=15, handlelength=0)
plt.tight_layout()  # Ajuste de layout para evitar corte da imagem
temp_img_filename = 'temp_pie.png'
plt.savefig(temp_img_filename)
plt.close()  # Fechar a figura para liberar memória

# Incluir o gráfico no arquivo MD
with open(md_filename, 'a', encoding='utf-8') as md_file:
    md_file.write('### Gráfico de Propostas Categorizadas\n\n')
    md_file.write('![Gráfico de Propostas Categorizadas](' + temp_img_filename + ')\n\n')

### ANÁLISE GERAL: Propostas por Ministério

In [55]:
# Código para plotar o segundo gráfico
TOP = 2
plt.figure(figsize=(10, 10))
df.groupby('category/name/pt-BR')['category/name/pt-BR'].count() \
    .sort_values(ascending=True).tail(40) \
    .plot.barh(title='Votos por Ministério',
               figsize=(10, 10),
               ylabel='',
               xlim=(0, 1333 + 100),
               color=[color.rgb for color in list(Color('white').range_to(Color(DARK_BLUE),40))],
               # add bar_label
               ).bar_label(container=plt.gca().containers[0], fmt=' %.0f')
plt.title('Quantidade de PROPOSTAS por Ministério', fontsize=16, fontweight='bold')
for i in range(TOP):
    plt.gca().get_yticklabels()[-i-1].set_fontweight('bold')
plt.tight_layout()  # Ajuste de layout para evitar corte da imagem
temp_img_filename2 = 'temp_bar.png'
plt.savefig(temp_img_filename2)
plt.close()  # Fechar a figura para liberar memória

# Adicionar o segundo gráfico ao arquivo MD
with open(md_filename, 'a', encoding='utf-8') as md_file:
    md_file.write('### Gráfico de Quantidade de PROPOSTAS por Ministério\n\n')
    md_file.write('![Gráfico de Quantidade de PROPOSTAS por Ministério](' + temp_img_filename2 + ')\n\n')

### ANÁLISE GERAL: Votos por Ministério

In [56]:
# Código para plotar o terceiro gráfico
TOP = 3
plt.figure(figsize=(10, 10))
df.groupby('category/name/pt-BR')['supports'].sum() \
    .sort_values(ascending=True).tail(40) \
    .plot.barh(title='Votos por Ministério',
               figsize=(10, 10),
               ylabel='',
               xlim=(0, 360000 + 50000),
               color=[color.rgb for color in list(Color('white').range_to(Color(DARK_GREEN),40))],
               # add bar_label
               ).bar_label(container=plt.gca().containers[0], fmt=' %.0f')
plt.title('Quantidade de VOTOS por Ministério', fontsize=16, fontweight='bold')
for i in range(TOP):
    plt.gca().get_yticklabels()[-i-1].set_fontweight('bold')
plt.tight_layout()  # Ajuste de layout para evitar corte da imagem
temp_img_filename3 = 'temp_bar_votos.png'
plt.savefig(temp_img_filename3)
plt.close()  # Fechar a figura para liberar memória

# Adicionar o terceiro gráfico ao arquivo MD
with open(md_filename, 'a', encoding='utf-8') as md_file:
    md_file.write('### Gráfico de Quantidade de VOTOS por Ministério\n\n')
    md_file.write('![Gráfico de Quantidade de VOTOS por Ministério](' + temp_img_filename3 + ')\n\n')


## Export Markdown